In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, col

StatementMeta(, 74b1d3c3-27fc-4dda-bda1-19346b2cff5b, 3, Finished, Available, Finished)

In [2]:
# 1️⃣ Read from Silver
orders = spark.read.format("delta").table("SilverLakehouse.dbo.slv_orders")
customers = spark.read.format("delta").table("SilverLakehouse.dbo.slv_customers")
products = spark.read.format("delta").table("SilverLakehouse.dbo.slv_products")

# 2️⃣ Join with dimensions
fact_orders = (
    orders.alias("o")
    .join(customers.alias("c"), col("o.Customer_ID") == col("c.Customer_ID"), "left")
    .join(products.alias("p"), col("o.Product_ID") == col("p.Product_ID"), "left")
    .select(
        col("o.Order_ID"),
        col("o.Customer_ID"),
        col("o.Product_ID"),
        col("o.Order_Date"),
        col("o.Quantity"),
        col("o.Price"),
        col("o.Order_Total"),
        col("o.Status"),
        col("p.Category").alias("Product_Category"),
        col("p.Brand").alias("Product_Brand"),
        col("c.Gender").alias("Customer_Gender"),
        col("c.Age").alias("Customer_Age"),
        col("o.ingestion_time")
    )
)


StatementMeta(, 74b1d3c3-27fc-4dda-bda1-19346b2cff5b, 4, Finished, Available, Finished)

In [3]:

# 3️⃣ Add gold ingestion timestamp
fact_orders = fact_orders.withColumn("gold_ingestion_time", current_timestamp())

# 4️⃣ Save as Fact Table
fact_orders.write.format("delta").mode("overwrite").saveAsTable("Fact_Orders")

StatementMeta(, 74b1d3c3-27fc-4dda-bda1-19346b2cff5b, 5, Finished, Available, Finished)